In [ ]:
import os
import json
import copy
from glob import glob
import matplotlib.pyplot as plt
import numpy as np
from pycalphad import calculate, equilibrium, Database, variables as v
from pycalphad.core.utils import filter_phases
from pycalphad.plot.utils import phase_legend
from scheil import simulate_scheil_solidification, simulate_equilibrium_solidification, SolidificationResult
from scheil.utils import order_disorder_dict, order_disorder_eq_phases
# local helpers.py file
# monkey patches pycalphad variables with weights, should be imported *after* pycalphad
from helpers import sample_phase_points, W_to_X, comp_interp 

Load the database file for this system

In [ ]:
dbf = Database('Fe-Ni-Ti_DeKeyzer.tdb')
comps = ['FE', 'NI', 'TI', 'VA']
phases = list(dbf.phases.keys())

In [ ]:
# Compute a map from ordered to disorderd phases, to determine equilibrium solidification phase amounts.
ord_disord_map = order_disorder_dict(dbf, comps, phases)

## Add points near the convex hull manually

In this system, the phases are modeled with many sublattices. Sampling over the single phase energy surfaces to find and sample phase constitutions around equilibrium composition sets can greatly increase the speed and accuracy of the solver during the solidification simulations.

Here, several temperatures are sampled.

In [ ]:
TEMPERATURES = [1400, 1660]
pts_dict = {}
active_phases = ['A3', 'BCC2', 'FCC4', 'NI3TI', 'NITI2', 'LIQUID', 'C14']
for phase_name in filter_phases(dbf, list(map(v.Species, comps)), phases):
    print(phase_name, end=': T= ')
    phase_pts = []
    for T in TEMPERATURES:
        print(T, end=' ')
        phase_pts.append(sample_phase_points(dbf, comps, phase_name, {v.T: T, v.P: 101325, v.N: 1, v.X('FE'): (0, 1.01, 0.1), v.X('NI'): (0, 1.01, 0.1)}, 500, 100))
    pts_dict[phase_name] = np.concatenate(phase_pts)
    print(f'- {pts_dict[phase_name].shape[0]} points')

# Individual layers

Note that in all simulations, the point dictionary will be copied. This means that each simulation will not be affected by others regardless of the number of times/order of the simulations. At the same time, the added points won't be able to accumulate through the simulations. They will be reproducible.

Common parameters for all simulations.

In [ ]:
RESULTS_DIR = os.path.join('results')
if not os.path.exists(RESULTS_DIR):
    os.mkdir(RESULTS_DIR)
LAYERS_DIR = os.path.join(RESULTS_DIR, 'layers')
if not os.path.exists(LAYERS_DIR):
    os.mkdir(LAYERS_DIR)

In [ ]:
step_temperature = 10.0
verbose = True

## CP Ti

### Layer 24

In [ ]:
composition = {v.W('NI'): 0.060, v.W('TI'): 0.834}
composition = W_to_X(composition, dbf, comps)
start_temperature = 1620
res = simulate_scheil_solidification(dbf, comps, phases, composition, start_temperature=start_temperature, step_temperature=step_temperature, verbose=verbose, eq_kwargs={'calc_opts': {'points': copy.deepcopy(pts_dict)}})
print(res)
with open(os.path.join(LAYERS_DIR, 'CPTi-L24.json'), 'w') as fp:
    json.dump(res.to_dict(), fp)
print(", ".join([f"({ph}: {vals[-1]:0.4f})" for ph, vals in res.cum_phase_amounts.items() if not np.isclose(vals[-1], 0)]))

In [ ]:
eq_sol = simulate_equilibrium_solidification(dbf, comps, phases, composition, start_temperature=start_temperature, step_temperature=step_temperature, verbose=verbose, eq_kwargs={'calc_opts': {'points': copy.deepcopy(pts_dict)}})
eq_res = equilibrium(dbf, comps, phases, {v.P: 101325, v.N: 1.0, v.T: eq_sol.temperatures[-1], **composition})
print(", ".join([f"({ph}: {val:0.4f})" for ph, val in zip(order_disorder_eq_phases(eq_res, ord_disord_map), eq_res.NP.values.squeeze()) if ph != '']))

### Layer 32

In [ ]:
composition = {v.W('NI'): 0.165, v.W('TI'): 0.533}
composition = W_to_X(composition, dbf, comps)
start_temperature = 1600
res = simulate_scheil_solidification(dbf, comps, phases, composition, start_temperature=start_temperature, step_temperature=step_temperature, verbose=verbose, eq_kwargs={'calc_opts': {'points': copy.deepcopy(pts_dict)}})
print(res)
with open(os.path.join(LAYERS_DIR, 'CPTi-L32.json'), 'w') as fp:
    json.dump(res.to_dict(), fp)
print(", ".join([f"({ph}: {vals[-1]:0.4f})" for ph, vals in res.cum_phase_amounts.items() if not np.isclose(vals[-1], 0)]))

In [ ]:
eq_sol = simulate_equilibrium_solidification(dbf, comps, phases, composition, start_temperature=start_temperature, step_temperature=step_temperature, verbose=verbose, eq_kwargs={'calc_opts': {'points': copy.deepcopy(pts_dict)}})
eq_res = equilibrium(dbf, comps, phases, {v.P: 101325, v.N: 1.0, v.T: eq_sol.temperatures[-1], **composition})
print(", ".join([f"({ph}: {val:0.4f})" for ph, val in zip(order_disorder_eq_phases(eq_res, ord_disord_map), eq_res.NP.values.squeeze()) if ph != '']))

## Ti-6Al-4V

### Layer 26

In [ ]:
composition = {v.W('NI'): 0.127, v.W('TI'): 0.647}
composition = W_to_X(composition, dbf, comps)
start_temperature = 1400
res = simulate_scheil_solidification(dbf, comps, phases, composition, start_temperature=start_temperature, step_temperature=step_temperature, verbose=verbose, eq_kwargs={'calc_opts': {'points': copy.deepcopy(pts_dict)}})
print(res)
with open(os.path.join(LAYERS_DIR, 'Ti64-L26.json'), 'w') as fp:
    json.dump(res.to_dict(), fp)
print(", ".join([f"({ph}: {vals[-1]:0.4f})" for ph, vals in res.cum_phase_amounts.items() if not np.isclose(vals[-1], 0)]))

In [ ]:
eq_sol = simulate_equilibrium_solidification(dbf, comps, phases, composition, start_temperature=start_temperature, step_temperature=step_temperature, verbose=verbose, eq_kwargs={'calc_opts': {'points': copy.deepcopy(pts_dict)}})
eq_res = equilibrium(dbf, comps, phases, {v.P: 101325, v.N: 1.0, v.T: eq_sol.temperatures[-1], **composition})
print(", ".join([f"({ph}: {val:0.4f})" for ph, val in zip(order_disorder_eq_phases(eq_res, ord_disord_map), eq_res.NP.values.squeeze()) if ph != '']))

### Layer 33

In [ ]:
composition = {v.W('NI'): 0.249, v.W('TI'): 0.328}
composition = W_to_X(composition, dbf, comps)
start_temperature = 1700
res = simulate_scheil_solidification(dbf, comps, phases, composition, start_temperature=start_temperature, step_temperature=step_temperature, verbose=verbose, eq_kwargs={'calc_opts': {'points': copy.deepcopy(pts_dict)}})
print(res)
with open(os.path.join(LAYERS_DIR, 'Ti64-L33.json'), 'w') as fp:
    json.dump(res.to_dict(), fp)
print(", ".join([f"({ph}: {vals[-1]:0.4f})" for ph, vals in res.cum_phase_amounts.items() if not np.isclose(vals[-1], 0)]))

In [ ]:
eq_sol = simulate_equilibrium_solidification(dbf, comps, phases, composition, start_temperature=start_temperature, step_temperature=step_temperature, verbose=verbose, eq_kwargs={'calc_opts': {'points': copy.deepcopy(pts_dict)}})
eq_res = equilibrium(dbf, comps, phases, {v.P: 101325, v.N: 1.0, v.T: eq_sol.temperatures[-1], **composition})
print(", ".join([f"({ph}: {val:0.4f})" for ph, val in zip(order_disorder_eq_phases(eq_res, ord_disord_map), eq_res.NP.values.squeeze()) if ph != '']))

### Layer 35

In [ ]:
composition = {v.W('NI'): 0.285, v.W('TI'): 0.235}
composition = W_to_X(composition, dbf, comps)
start_temperature = 1700
res = simulate_scheil_solidification(dbf, comps, phases, composition, start_temperature=start_temperature, step_temperature=step_temperature, verbose=verbose, eq_kwargs={'calc_opts': {'points': copy.deepcopy(pts_dict)}})
print(res)
with open(os.path.join(LAYERS_DIR, 'Ti64-L35.json'), 'w') as fp:
    json.dump(res.to_dict(), fp)
print(", ".join([f"({ph}: {vals[-1]:0.4f})" for ph, vals in res.cum_phase_amounts.items() if not np.isclose(vals[-1], 0)]))

In [ ]:
eq_sol = simulate_equilibrium_solidification(dbf, comps, phases, composition, start_temperature=start_temperature, step_temperature=step_temperature, verbose=verbose, eq_kwargs={'calc_opts': {'points': copy.deepcopy(pts_dict)}})
eq_res = equilibrium(dbf, comps, phases, {v.P: 101325, v.N: 1.0, v.T: eq_sol.temperatures[-1], **composition})
print(", ".join([f"({ph}: {val:0.4f})" for ph, val in zip(order_disorder_eq_phases(eq_res, ord_disord_map), eq_res.NP.values.squeeze()) if ph != '']))

#### Visualizations

##### Phase fractions through solidification

In [ ]:
for phase_name, amounts in res.cum_phase_amounts.items():
    if amounts[-1] > 0:  # If the total phase fraction is non-zero
        plt.plot(res.temperatures, amounts, label=phase_name)
plt.legend()
plt.xlabel('Temperature (K)')
plt.ylabel('Phase Fraction')

##### Solidification path

In [ ]:
fig = plt.figure(figsize=(8,6))
ax = fig.gca(projection='triangular')
ax.plot(res.x_liquid['NI'], res.x_liquid['TI'])
ax.scatter([res.x_liquid['NI'][0]], [res.x_liquid['TI'][0]], label='Start', zorder=10)
ax.scatter([res.x_liquid['NI'][-2]], [res.x_liquid['TI'][-2]], label='End', zorder=10)
ax.set_xlabel('X(NI)', size=15)
ax.set_ylabel('X(TI)', rotation=60, labelpad=-80, size=15)
ax.set_title('Solidification Path', size=20)
ax.legend()

### Layer 45

In [ ]:
composition = {v.W('NI'): 0.332, v.W('TI'): 0.120}
composition = W_to_X(composition, dbf, comps)
start_temperature = 1540
res = simulate_scheil_solidification(dbf, comps, phases, composition, start_temperature=start_temperature, step_temperature=step_temperature, verbose=verbose, eq_kwargs={'calc_opts': {'points': copy.deepcopy(pts_dict)}})
print(res)
with open(os.path.join(LAYERS_DIR, 'Ti64-L45.json'), 'w') as fp:
    json.dump(res.to_dict(), fp)
print(", ".join([f"({ph}: {vals[-1]:0.4f})" for ph, vals in res.cum_phase_amounts.items() if not np.isclose(vals[-1], 0)]))

In [ ]:
eq_sol = simulate_equilibrium_solidification(dbf, comps, phases, composition, start_temperature=start_temperature, step_temperature=step_temperature, verbose=verbose, eq_kwargs={'calc_opts': {'points': copy.deepcopy(pts_dict)}})
eq_res = equilibrium(dbf, comps, phases, {v.P: 101325, v.N: 1.0, v.T: eq_sol.temperatures[-1], **composition})
print(", ".join([f"({ph}: {val:0.4f})" for ph, val in zip(order_disorder_eq_phases(eq_res, ord_disord_map), eq_res.NP.values.squeeze()) if ph != '']))

# Gradient Path Solidification
## Path construction

Build a composition path from pure Ti to Invar-36 (Fe-36 wt% Ni). Weight fractions are converted to mole fractions (as used by pycalphad).

Scheil-Gulliver and equilibrium solidification simulations will loop over this linear path.

In [ ]:
start = {v.W('NI'): 0.0, v.W('TI'): 1.0} # pure Ti
end = {v.W('NI'): 0.36, v.W('TI'): 0.0}  # Invar-36
def fmt_comps(comp_conds):
    return '-'.join([f'{c}={val:0.3f}' for c, val in comp_conds.items()])

all_comps = []  # mole fraction
for mix in np.linspace(1e-3, 1, 50):
    all_comps.append(W_to_X(comp_interp(start, end, mix), dbf, comps))

# Note, compositions here are in mole fraction, but input were mass fraction.
print(f"Start: {fmt_comps(all_comps[0])}")
print(f"End  : {fmt_comps(all_comps[-1])}")

## Scheil-Gulliver
### Settings

Save the results in a local results directory and set the main controlling.

The start temperature should be the highest temperature that is pure liquid along the whole path. Titanium has a melting point of about 1940 K, so 2000 K is chosen.

In [ ]:
RESULTS_DIR = os.path.join('results')
if not os.path.exists(RESULTS_DIR):
    os.mkdir(RESULTS_DIR)
PATH_DIR = os.path.join(RESULTS_DIR, 'scheil-path')
if not os.path.exists(PATH_DIR):
    os.mkdir(PATH_DIR)
START_TEMPERATURE = 2000.0  # global start temperature
STEP_TEMPERATURE = 10.0  # temperature step size
VERBOSE = False

### Simulations

Typically each simulation on the path finishes in ~7 minutes on average for a 2015 MacBook Pro with an Intel i7 processor.
50 compositions gives a total time of about 6 hours.

In [ ]:
prefix = 'Fe-Ni-Ti_path'  # file name prefix
# copy points at first for reproducibilty, but use along the whole path for speed and accuracy
path_pts = copy.deepcopy(pts_dict)
for i, composition in enumerate(all_comps):
    formatted_name = fmt_comps(composition)
    print(formatted_name, end='  ==  ')
    res = simulate_scheil_solidification(dbf, comps, phases, composition, start_temperature=START_TEMPERATURE, adaptive=True, step_temperature=STEP_TEMPERATURE, verbose=VERBOSE, eq_kwargs={'calc_opts': {'points': path_pts}})
    for ph, amnts in res.cum_phase_amounts.items():
        print(f'{ph}: {amnts[-1]:0.4f}', end=', ')
    print()
    with open(os.path.join(PATH_DIR, f'{prefix}_i={i:03d}_{formatted_name}.json'), 'w') as fp:
        json.dump(res.to_dict(), fp)

### Plotting
Load the data for the path from the results directory and create a dictionary mapping a phase to the fractions along the path.

In [ ]:
MIN_PHASE_FRACTION = 1e-6
solid_phases = [ph for ph in phases if ph != 'LIQUID']
phase_fractions = {ph: [] for ph in solid_phases if ph != solid_phases}
for filename in sorted(glob(os.path.join(f'{PATH_DIR}', '*'))):
    with open(filename) as fp:
        data = json.load(fp)
    sol_res = SolidificationResult.from_dict(data)
    for ph in solid_phases:
        phase_fractions[ph].append(sol_res.cum_phase_amounts[ph][-1])
# Only include phases that have *some* amount greater than MIN_PHASE_FRACTION
phase_fractions = {ph: np.array(fracs) for ph, fracs in phase_fractions.items() if np.max(fracs) > MIN_PHASE_FRACTION}

Manually set the line styles and coloring.

In [ ]:
LINE_STYLES = {
    'A1': 'dotted',
    'A2': (0, (3, 5, 1, 5)),
    'A3': (0, (3, 10, 1, 10)),
    'BCC2': (0, (5, 5)),
    'C14': 'dashed',
    'FCC4': (0, (5, 10)),
    'NI3TI': 'solid',
    'NITI2': 'dashdot'
}
COLORS = {
    'A1': '#000000',
    'A2': '#000075',
    'A3': '#808000',
    'BCC2': '#911eb4',
    'C14': '#f58231',
    'FCC4': '#800000',
    'NI3TI': '#3cb44b',
    'NITI2': '#9A6324'
}

Plot the phase fractions.

In [ ]:
path_xs = np.linspace(0, 1, list(phase_fractions.values())[0].size) # plotting along the fraction of Invar
fig = plt.figure()
ax = fig.gca()
for ph, fracs in phase_fractions.items():
    ax.plot(path_xs, fracs, color=COLORS[ph], ls=LINE_STYLES[ph], label=ph)


ax.set_xlabel('Fraction Invar', fontsize=14)
ax.set_ylabel('Phase Fraction\nScheil Solidification', fontsize=14)
#ax.legend(legend_handles, {ph: COLORS[ph] for ph in sorted(phase_fractions.keys())}, loc=(1.02,0.2))
ax.legend(loc=(1.02,0.2), handlelength=4)
ax.set_xlim(0, 1)
ax.set_ylim(-0.01, 1.01)
fig.savefig('scheil-gulliver-path-solidification.tiff', bbox_inches='tight')

## Calculating equilibrium solidification phase fraction 

### Settings

Since the phase fractions we are interested in plotting are at the end of the simulations and the equilibrium solidification simulations do a binary search at the end, we don't need as fine of a step size.

In [ ]:
RESULTS_DIR = os.path.join('results')
if not os.path.exists(RESULTS_DIR):
    os.mkdir(RESULTS_DIR)
PATH_DIR = os.path.join(RESULTS_DIR, 'eq-path')
if not os.path.exists(PATH_DIR):
    os.mkdir(PATH_DIR)
START_TEMPERATURE = 2000.0  # global start temperature
STEP_TEMPERATURE = 10.0  # temperature step size
VERBOSE = False

### Simulations

Typically each simulation on the path finishes in ~4 minutes on average for a 2015 MacBook Pro with an Intel i7 processor.

In [ ]:
prefix = 'Fe-Ni-Ti_path'
# copy points at first for reproducibilty, but use along the whole path for speed
path_pts = copy.deepcopy(pts_dict)
for i, composition in enumerate(all_comps):
    formatted_name = fmt_comps(composition)
    print(formatted_name, end='  ==  ')
    res = simulate_equilibrium_solidification(dbf, comps, phases, composition, start_temperature=START_TEMPERATURE, adaptive=True, step_temperature=STEP_TEMPERATURE, verbose=VERBOSE, eq_kwargs={'calc_opts': {'points': path_pts}})
    for ph, amnts in res.cum_phase_amounts.items():
        print(f'{ph}: {amnts[-1]:0.4f}', end=', ')
    print()
    with open(os.path.join(PATH_DIR, f'{prefix}_i={i:03d}_{formatted_name}.json'), 'w') as fp:
        json.dump(res.to_dict(), fp)

In [ ]:
MIN_PHASE_FRACTION = 1e-6
solid_phases = [ph for ph in phases if ph != 'LIQUID']
phase_fractions = {ph: [] for ph in solid_phases if ph != solid_phases}
for filename in sorted(glob(os.path.join(f'{PATH_DIR}', '*'))):
    with open(filename) as fp:
        data = json.load(fp)
    sol_res = SolidificationResult.from_dict(data)
    for ph in solid_phases:
        phase_fractions[ph].append(sol_res.cum_phase_amounts[ph][-1])
# Only include phases that have *some* amount greater than MIN_PHASE_FRACTION
phase_fractions = {ph: np.array(fracs) for ph, fracs in phase_fractions.items() if np.max(fracs) > MIN_PHASE_FRACTION}

In [ ]:
# Same styles and colors as in the Scheil-Gulliver path plot
LINE_STYLES = {
    'A1': 'dotted',
    'A2': (0, (3, 5, 1, 5)),
    'A3': (0, (3, 10, 1, 10)),
    'BCC2': (0, (5, 5)),
    'C14': 'dashed',
    'FCC4': (0, (5, 10)),
    'NI3TI': 'solid',
    'NITI2': 'dashdot'
}
COLORS = {
    'A1': '#000000',
    'A2': '#000075',
    'A3': '#808000',
    'BCC2': '#911eb4',
    'C14': '#f58231',
    'FCC4': '#800000',
    'NI3TI': '#3cb44b',
    'NITI2': '#9A6324'
}

In [ ]:
path_xs = np.linspace(0, 1, list(phase_fractions.values())[0].size) # plotting along the fraction of Invar
fig = plt.figure()
ax = fig.gca()
for ph, fracs in phase_fractions.items():
    ax.plot(path_xs, fracs, color=COLORS[ph], ls=LINE_STYLES[ph], label=ph)


ax.set_xlabel('Fraction Invar', fontsize=14)
ax.set_ylabel('Phase Fraction\nEquilibrium Solidification', fontsize=14)
ax.legend(loc=(1.02,0.2), handlelength=4)
ax.set_xlim(0, 1)
ax.set_ylim(-0.01, 1.01)
fig.savefig('eq-path-solidification.tiff', bbox_inches='tight')